In [27]:
import tensorrt as trt
import numpy as np
import onnx

In [28]:
ONNX_PATH = '/home/users/xt37/zz/onnx/best.onnx'
TRT_PATH = '/home/users/xt37/zz/trt/best.trt'

In [29]:
model_onnx = onnx.load(ONNX_PATH)

In [31]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network()

# create onnx-tensorrt backend
parser = trt.OnnxParser(network, builder.logger)
parser.parse(model_onnx.SerializeToString())


True

In [48]:
# set up optimization profile and build parameters
profile = builder.create_optimization_profile()

input_name = model_onnx.graph.input[0].name
min_shape = (1, 3, 640, 640)
opt_shape = (1, 3, 640, 640)
max_shape = (1, 3, 640, 640)

profile.set_shape(input_name, min_shape, opt_shape, max_shape)

In [49]:
builder_config = builder.create_builder_config()

In [52]:
builder_config.flags = 1 << int(trt.BuilderFlag.FP16)

In [ ]:
engine = builder.build_engine(network, builder_config)